## Tugas 5 Modeling LSA

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_csv(
    'https://raw.githubusercontent.com/Tegar11-F/Data-Set/main/datacomment1.csv')
df


,comment,comment (clean)
0,"ID\nAbaikan navigasi\nLogin\nPemilu 2024, Nama...",id abaikan navigasi login pemilu nama anies ga...
1,Saya heran sebenarnya siapa yg mau calonkan Ga...,saya heran sebenarnya siapa yang mau calonkan ...
2,Tetap amanah dalam mengawal Bangsa sebagai sum...,tetap amanah dalam mengawal bangsa sebagai sum...
3,Semoga negara kita dpt pemimipin dan wakilnya ...,semoga negara kita dapat pemimipin dan wakilny...
4,Insya Allah penyambung lidah rakyat ketahanan ...,insya allah penyambung lidah rakyat ketaha pan...
...,...,...
356,Ganjar -prabowo tiada lawan istebes,ganjar prabowo tiada lawan istebes
357,Gmn ceritanya?pak ganjar belum di usung sm PDI...,bagaimana ceritanya pak ganjar belum di usung ...
358,Tiga tiganya ga ada yg tak pilih.. berarti ga ...,tiga tiganya enggak ada yang tak pilih berarti...
359,Ganjar,ganjar


## Modeling

In [3]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [4]:
True

True

In [5]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                             stop_words=stopwords,
                             tokenizer=tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

# Bobot kata

In [6]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('ganjar', 0.9252800834328172), ('prabowo', 0.24611795076284412)]
Topic 1:  [('prabowo', 0.8774266321383564), ('maju', 0.10944081737860746)]
Topic 2:  [('pranowo', 0.6947121842247836), ('anis', 0.2893949637924045)]
Topic 3:  [('anis', 0.5789055993819835), ('baswedan', 0.3187459733778684)]


In [7]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=[
                      "Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik'] = df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(
    lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id abaikan navigasi login pemilu nama anies ga...,0.215361,0.087568,0.139409,0.104762,0
1,saya heran sebenarnya siapa yang mau calonkan ...,0.106084,-0.020983,-0.025342,0.043462,0
2,tetap amanah dalam mengawal bangsa sebagai sum...,0.003124,0.007114,0.023898,0.023218,2
3,semoga negara kita dapat pemimipin dan wakilny...,0.043136,0.067122,0.156414,0.063189,2
4,insya allah penyambung lidah rakyat ketaha pan...,0.017899,0.017536,0.076729,0.030046,2
...,...,...,...,...,...,...
356,ganjar prabowo tiada lawan istebes,0.229261,0.214121,-0.018656,-0.045061,0
357,bagaimana ceritanya pak ganjar belum di usung ...,0.104697,-0.015138,-0.017301,0.036971,0
358,tiga tiganya enggak ada yang tak pilih berarti...,0.032655,-0.010816,0.048003,0.056361,3
359,ganjar,0.925280,-0.194693,-0.274212,0.084712,0


In [8]:
df_lsa['Topik'].value_counts()

0    148
3    102
2     70
1     41
Name: Topik, dtype: int64